In [ ]:
from bg_atlasapi import BrainGlobeAtlas
from pprint import pprint
# this downloads the atlas and meshes,we could also do it straight from the allen but this is easier.
bg_atlas = BrainGlobeAtlas("allen_mouse_25um", check_latest=False)  

In [ ]:
from tifffile import imread
import meshio
import json
from pathlib import Path
# load the atlas metadata
atlaslocation =  Path('~').expanduser()/'.brainglobe'/'allen_mouse_25um_v1.2/'
with open(atlaslocation/'structures.json','r') as fd:
    structures = json.load(fd)
with open(atlaslocation/'metadata.json','r') as fd:
    metadata = json.load(fd)

with open(atlaslocation/'structures.json','r') as fd:
    structures = json.load(fd)

# AP,DV,ML
bregma_location = np.array([216, 18,228 ])*metadata['resolution'] #don't trust too much, for the 25um atlas from https://community.brain-map.org/t/how-to-transform-ccf-x-y-z-coordinates-into-stereotactic-coordinates/1858

#bregma_location = [540, 44, 570] #don't trust too much, for the 10um atlas from https://community.brain-map.org/t/how-to-transform-ccf-x-y-z-coordinates-into-stereotactic-coordinates/1858



import pandas as pd
structures = pd.DataFrame(structures)

# load meshes
def load_structure_mesh(atlaslocation,structures,acronym):
    # meshes are in um
    id = structures[structures.acronym == acronym].id.values
    if len(id):
        id = id[0]
    else:
        return
    mesh = atlaslocation/'meshes'/f'{id}.obj'
    mesh = pv.read(mesh)
    return mesh, structures[structures.acronym == acronym].iloc[0]
root,rootstruct = load_structure_mesh(atlaslocation,structures,'root')

toplot = []
for acronym in ['CP','LP','VISp','VISa','SCs','ACA','MD']:
    toplot.append(load_structure_mesh(atlaslocation,structures,acronym))

# todo, translate and rotate the meshes so they are in stereotaxic position.

import pyvista as pv # for plotting 3d
pv.set_jupyter_backend('trame')
#p = pv.Plotter(window_size = [1000,1000])# for saving in the next cell with high res
p = pv.Plotter(window_size = [600,400])

axes = p.add_axes()
axes.origin = bregma_location

rotate5deg = True
if rotate5deg:
    root.rotate_z(5,point=axes.origin, inplace=True)

# There are 2 ways of doing this, transform the meshes of transform the probes, lets transform the probes,
# the other way silhoette doesnt work.
# M = np.array([[0.025, 0., 0., 0.],
#                    [0., 0.025, 0., 0.],
#                    [0., 0., 0.025, 0.],
#                    [0., 0., 0., 1.]])
p.add_mesh(root, color = rootstruct['rgb_triplet'], opacity = 0.1,silhouette=False)#,user_matrix=M)
for s in toplot:
    if rotate5deg:
        s[0].rotate_z(5,point=axes.origin, inplace=True)
    p.add_mesh(s[0],color=s[1]['rgb_triplet'],opacity = 0.7, silhouette=dict(color='#000000',line_width=1) )

# todo, make all probes start at the same depth (needs angles)
probes_position_type = [(np.array([-2394,781,-991]),np.array([-1278,2197,-192]),4),
                        (np.array([-1665,-646,1898]),np.array([1210,3664,-1022]),4),
                        (np.array([131,646,-4224]),np.array([431,4235,-2819]),4),
                        (np.array([131,646,-4224]),np.array([431,4235,-2819]),4),
                        (np.array([1288,-103,-1132]),np.array([1862,3472,-1286]),4),
                        (np.array([4215,-650,-2241]),np.array([2628,3681,-1501]),4),
                        (np.array([3489,292,1176]),np.array([3489,2074,-1377]),4)]

for start,tip,nshanks in probes_position_type:
    # Can only plot start and end locations
    line = pv.Line(pointa = start+bregma_location, pointb = tip+bregma_location)
    p.add_mesh(line,color='#000000',opacity = 0.9,line_width=3)

p.show()


In [ ]:
p.save_graphic('/home/joao/Desktop/test.pdf')
# start,tip = (np.array([-1665,-646,1898]),np.array([1210,3664,-1022]))





In [ ]:
p.add_axes?